In [ ]:
import numpy as np
import flopy as fp
import matplotlib.pyplot as plt
%matplotlib inline
import os
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
model_dir = 'theis_flopy_mf6'
model_name = 'theis'
if not os.path.exists(model_dir):
    os.mkdir(model_dir)

In [ ]:
# create simulation
sim = fp.mf6.MFSimulation(sim_name='theis_mf6', version='mf6', exe_name='mf6', 
                             sim_ws=model_dir)


In [ ]:
# create tdis package
tdis_rc = [(86400.0, 75, 1.2)]
tdis = fp.mf6.ModflowTdis(sim, pname='tdis', time_units='DAYS', 
                             perioddata=tdis_rc)

In [ ]:
# create gwf model
gwf = fp.mf6.ModflowGwf(sim, modelname=model_name,
                           model_nam_file='{}.nam'.format(model_name))
gwf.name_file.save_flows = True

In [ ]:
# create iterative model solution and register the gwf model with it
ims = fp.mf6.ModflowIms(sim, pname='ims', print_option='SUMMARY', 
                           complexity='SIMPLE', outer_dvclose=1.e-2, 
                           outer_maximum=10, under_relaxation='NONE', 
                           inner_maximum=10, inner_dvclose=1.e-4, 
                           rcloserecord=0.1, linear_acceleration='BICGSTAB', 
                           scaling_method='NONE', reordering_method='NONE', 
                           relaxation_factor=0.99)

In [ ]:
sim.register_ims_package(ims, [gwf.name])


In [ ]:
# dis
nlay = 1
nrow = 1001
ncol = 1001
delr = 100
delc = 100
top = 0
botm = -1
dis = fp.mf6.ModflowGwfdis(gwf, nlay=nlay, nrow=nrow, ncol=ncol,
                              delr=delr, delc=delc,
                              top=top, botm=botm)

In [ ]:
# initial conditions
ic = fp.mf6.ModflowGwfic(gwf, pname='ic', strt=0.)

In [ ]:
# CHD equal to 0 at the boundaries
chd_spd = [[(0,0,i),0] for i in range(1,ncol)] + \
          [[(0,i,0),0] for i in range(nrow)] + \
          [[(0,nrow-1,i),0] for i in range(1,ncol)] + \
          [[(0,i,ncol-1),0] for i in range(1,nrow-1)]
chd_spd[:10]

In [ ]:
chd = fp.mf6.ModflowGwfchd(gwf, save_flows=True,stress_period_data = chd_spd)

In [ ]:
# npf 
hk=0.3

# node property flow
npf = fp.mf6.ModflowGwfnpf(gwf, save_specific_discharge=True,
                              icelltype=[0], 
                              k=[hk])

In [ ]:
# storage
ss=0.0008
stor = fp.mf6.ModflowGwfsto(gwf, ss=ss, transient=True)

In [ ]:
# single well in the center
well_sp = [[(0, 500, 500), -1.16]]
wel = fp.mf6.ModflowGwfwel(gwf,save_flows=True,stress_period_data=well_sp)

In [ ]:
# output control
oc = fp.mf6.ModflowGwfoc(gwf, pname='oc', budget_filerecord='{}.cbc'.format(model_name),
                            head_filerecord='{}.hds'.format(model_name),
                            headprintrecord=[('COLUMNS', 10, 'WIDTH', 15,
                                              'DIGITS', 6, 'GENERAL')],
                            saverecord=[('HEAD', 'ALL'), ('BUDGET', 'ALL')],
                            printrecord=[('HEAD', 'ALL'), ('BUDGET', 'ALL')])

In [ ]:
sim.write_simulation()
sim.run_simulation()

In [ ]:
head = gwf.output.head().get_alldata()
head.shape

In [ ]:
with PdfPages('theis_mf.pdf') as pdfout:
    for i, chead in enumerate(head):
        printhead = np.squeeze(chead)
        plt.figure(figsize=(8,8))
        plt.imshow(-printhead[490:510,490:510],
                  interpolation='nearest',
                  vmin=0,
                  vmax=2.7)
        plt.colorbar()
        plt.title('Theis-o-matic --> timestep{0}'.format(i))
        pdfout.savefig()
        plt.close()
        

In [ ]:
import theis_script2

In [ ]:
hydrograph = [head[:,0,500,501],head[:,0,500,510]]

In [ ]:
alltimes = gwf.output.head().get_times()

In [ ]:
plt.plot(alltimes,hydrograph[0])
plt.plot(alltimes,hydrograph[1])

In [ ]:
import theis_script2

In [ ]:
dd_analytical = theis_script2.theis_analytical(alltimes)

In [ ]:
if os.path.exists('tmp_images'):
    shutil.rmtree('tmp_images')
os.mkdir('tmp_images')
# make a temporary directory for the images.

In [ ]:
with PdfPages('theis_modflow_analytical.pdf') as pdfout:
    for i in range(len(alltimes)):
        fig = plt.figure(figsize=(8,8))
        fig.add_subplot(121)
        plt.imshow(-head[i,0,490:510,490:510],
                  interpolation='nearest',
                  vmin = 0,
                  vmax = 1.7, cmap='magma')
        plt.title('MODFLOW')
        fig.add_subplot(122)
        plt.imshow(dd_analytical[i], interpolation='nearest',
                  vmin=0,
                  vmax=1.7, cmap='magma')
        plt.title('Analytical')
        plt.suptitle('Theis two ways: timestep = {0}'.format(i+1))
        plt.tight_layout()
        
        pdfout.savefig()
        
         # also make a png
        plt.savefig('tmp_images/{0}.png'.format(i))
        
        plt.close()

In [ ]:
# make a gif
import imageio

gif_path = 'modflow_theis.gif'
frames_path = 'tmp_images/{i}.png'

with imageio.get_writer(gif_path, mode='I') as writer:
    for i in range(len(alltimes)):
        writer.append_data(imageio.imread(frames_path.format(i=i)))

In [ ]:
# make a .mpeg
import imageio_ffmpeg

mp4_path = 'modflow_theis.mp4'
frames_path = 'tmp_images/{i}.png'

with imageio.get_writer(mp4_path, mode='I') as writer:
    for i in range(len(alltimes)):
        writer.append_data(imageio.imread(frames_path.format(i=i)))